In [1]:
import torch
import sys
import esm

In [2]:
model_data  = torch.load('./pretrained_model/esm1_t6_43M_UR50S.pt', map_location="cpu")
model,alphabet = esm.pretrained.load_model_and_alphabet_core(model_data, None)
batch_converter = alphabet.get_batch_converter()

/data/zhangruochi/projects/pseudoESM/esm/pretrained.py:174: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


In [4]:
# for id,layer in enumerate(model.layers):
#     print('{},***{}'.format(id,layer))

In [6]:
model.eval()  # disables dropout for deterministic results

# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
data = [
    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
    ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
    ("protein2 with mask","KALTARQQEVFDLIRD<mask>ISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
    ("protein3",  "K A <mask> I S Q"),
]

batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[6], return_contacts=True)

In [20]:
results.keys()

dict_keys(['logits', 'representations', 'attentions', 'contacts'])

In [22]:
results["contacts"].shape, results["representations"][6].shape, results[
    "logits"].shape, results["attentions"].shape


(torch.Size([4, 71, 71]),
 torch.Size([4, 72, 768]),
 torch.Size([4, 72, 35]),
 torch.Size([4, 6, 12, 72, 72]))